In [ ]:
%cd /content/drive/MyDrive/Assignment_2

/content/drive/MyDrive/Assignment_2


In [ ]:
!pip install libsvm

     |████████████████████████████████| 174kB 5.9MB/s 
  Created wheel for libsvm: filename=libsvm-3.23.0.4-cp36-cp36m-linux_x86_64.whl size=233333 sha256=7346c32729c4b130195775be677fffd3e1e5e5ce1aa52cd062d75c3dbb5d02fd
  Stored in directory: /root/.cache/pip/wheels/5e/9e/b5/dbb033107407eec2f52b8cd24cf024a4b9ec8b62ea5aee995a
Successfully built libsvm


In [ ]:
from libsvm.svmutil import *
import os
import numpy as np
import csv
import random
import matplotlib.pyplot as plt
from sklearn import svm

In [ ]:
#convert data to SVM format
train_x=[]
train_y=[]
test_x=[]
test_y=[]
n=0

train_idx=random.sample(range(700),400)
with open('health_data.csv','r') as file:
  csvf= csv.DictReader(file)
  data=[]
  for row in csvf:
    data=list(row.values())
    if n in train_idx:
      train_x.append([int(i)/80 for i in data[:-1]])
      train_y.append(1 if int(data[-1])==1 else -1)
    else:
      test_x.append([int(i)/80 for i in data[:-1]])
      test_y.append(1 if int(data[-1])==1 else -1)
    n+=1
test_x=np.array(test_x)
train_x=np.array(train_x)
test_y=np.array(test_y)
train_y=np.array(train_y)


In [ ]:
#calculates w and b for a given svm model
def calculate_params(m):
  Support_vectors=m.get_SV()
  w=np.zeros((len(Support_vectors[0])-1,1))
  b=m.rho[0]
  n_sv=len(Support_vectors)
  lagrange_multipliers=m.get_sv_coef()
  for i in range(n_sv):
    for j in range(w.shape[0]):
      w[j]+=Support_vectors[i][j+1]*lagrange_multipliers[i][0]
  return w,b

In [ ]:
#Linear C-SVM with different C values. 3D data
C=[0.001,0.03,0.1,0.6,1,30,600,1000]
prob  = svm_problem(train_y, train_x,isKernel=False)
temp=-1
maxi=-1
best_model=None
for i in C:
  param = svm_parameter('-t 0 -c '+str(i))
  m = svm_train(prob, param)
  p_label_train,p_acc_train,p_val_train=svm_predict(train_y,train_x,m)
  ACC_train,s,sl=evaluations(train_y,p_label_train)
  p_label, p_acc, p_val = svm_predict(test_y, test_x, m)
  ACC, MSE, SCC = evaluations(test_y, p_label)
  print(ACC,ACC_train)
  if(ACC>=temp):
    temp=ACC
    maxi=i
    best_model=m

w,b=calculate_params(best_model)


In [ ]:
#polynomial SVM with specified degrees and different C values
C=[0.001,0.03,0.1,0.6,1,30,600,1000]
deg=[2,3,4]
best_models=[]
lookup=dict()
prob  = svm_problem(train_y, train_x,isKernel=False)
for d in deg:
  temp=-1
  maxi=-1
  best_model=None
  lookup[d]=[]
  for i in C:
    param = svm_parameter('-t 1  -c '+str(i)+' -d '+str(d))
    m = svm_train(prob, param)
    p_label_train,p_acc_train,p_val_train=svm_predict(train_y,train_x,m)
    ACC_train,s,sl=evaluations(train_y,p_label_train)
    p_label, p_acc, p_val = svm_predict(test_y, test_x, m)
    ACC, MSE, SCC = evaluations(test_y, p_label)
    print(ACC,ACC_train)
    lookup[d].append((i,ACC,ACC_train,m))
    if(ACC>=temp):
      temp=ACC
      maxi=i
      best_model=m
  best_models.append(best_model)



In [ ]:
#RBF SVM with grid search on C and gamma
C=[0.001,0.03,0.1,0.6,1,30,600,1000]
gamma=[0.001,0.05,0.5,1,50,500,1000]
best_models=[]
prob  = svm_problem(train_y, train_x,isKernel=False)
for g in gamma:
  temp=-1
  maxi=-1
  best_model=None
  for i in C:
    param = svm_parameter('-t 2  -c '+str(i)+' -g '+str(g))
    m = svm_train(prob, param)
    p_label_train,p_acc_train,p_val_train=svm_predict(train_y,train_x,m)
    ACC_train,s,sl=evaluations(train_y,p_label_train)
    p_label, p_acc, p_val = svm_predict(test_y, test_x, m)
    ACC, MSE, SCC = evaluations(test_y, p_label)
    print(ACC,ACC_train)
    if(ACC>=temp):
      temp=ACC
      maxi=i
      best_model=m
  best_models.append(best_model)

In [ ]:
def PCA(X , k):    
    X_shifted = X - np.mean(X , axis = 0)
    sigma = np.cov(X_shifted , rowvar = False)
    eigen_values , eigen_vectors = np.linalg.eigh(sigma)
    sorted_index = np.argsort(eigen_values)[::-1]
    sorted_eigenvalue = eigen_values[sorted_index]
    sorted_eigenvectors = eigen_vectors[:,sorted_index]
    eigenvector_subset = sorted_eigenvectors[:,0:k]
    X_reduced = np.dot(eigenvector_subset.transpose() , X_shifted.transpose() ).transpose()
    return X_reduced

In [ ]:
#reduced dimension of data to 2 so we can plot it.
train_x_p=PCA(train_x,2)

test_x_p=PCA(test_x,2)

In [ ]:
#removes support vectors from dataset
def remove_SV(sv_list,train_x,train_y):
  mm=[]
  yy=[]
  for i in range(400):
    if list(train_x[i,:]) not in sv_list:
      mm.append(train_x[i,:])
      yy.append(train_y[i])
  print(hist(yy))
  return mm,yy

In [ ]:
#removes support vectors. change value of m to use it for different models.
m=best_models[0]
svs=m.get_SV()
sv_list=[]
for i in range(len(svs)):
  sv_list.append(list(svs[i].values())[:-1])
train_x_l,train_y_l=remove_SV(sv_list,train_x_p,train_y)
print(len(train_x_l))
print(len(svs))

In [ ]:
#Linear C-SVM with different C values. 2D data
C=[0.001,0.03,0.6,0.1,1,30,600,1000]
prob  = svm_problem(train_y_l, train_x_l,isKernel=False)
temp=-1
maxi=-1
best_model=None
for i in C:
  param = svm_parameter('-t 0 -c '+str(i))
  m = svm_train(prob, param)
  p_label_train,p_acc_train,p_val_train=svm_predict(train_y,train_x_p,m)
  ACC_train,s,sl=evaluations(train_y,p_label_train)
  p_label, p_acc, p_val = svm_predict(test_y, test_x_p, m)
  ACC, MSE, SCC = evaluations(test_y, p_label)
  print(ACC,ACC_train)
  if(ACC>=temp):
    temp=ACC
    maxi=i
    best_model=m

w,b=calculate_params(best_model)


In [ ]:
# plotting the data
h=0.1
i=2
# m=best_model
# train_x_l=np.array(train_x_l)
# train_y_l=np.array(train_y_l)
x_min, x_max = train_x_p[:, 0].min() - 1, train_x_p[:, 0].max() + 1
y_min, y_max = train_x_p[:, 1].min() - 1, train_x_p[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

Z,x,x=svm_predict(np.ones(xx.ravel().shape[0]),np.c_[xx.ravel(),yy.ravel()],m)
Z=np.array(Z)
Z=Z.reshape(xx.shape)
plt.figure(figsize=(7,5))
plt.contourf(xx,yy,Z,cmap=plt.cm.coolwarm,alpha=0.8)
plt.scatter(negative[:, 0], negative[:, 1], cmap=plt.cm.coolwarm,color='blue',marker='o',s=20)
plt.scatter(positive[:, 0], positive[:, 1], cmap=plt.cm.coolwarm,color='yellow',marker='x',s=20)


plt.scatter(negative_sv[:,0],negative_sv[:,1],s=20,marker="o",color="pink",cmap=plt.cm.coolwarm)
plt.scatter(positive_sv[:,0],positive_sv[:,1],s=20,marker="x",color="green",cmap=plt.cm.coolwarm)


In [ ]:
#polynomial SVM with specified degrees and different C values-2D data
C=[0.001,0.03,0.6,0.1,1,30,600,1000]
deg=[2,3,4]
best_models=[]
prob  = svm_problem(train_y_l, train_x_l,isKernel=False)
for d in deg:
  temp=-1
  maxi=-1
  print(d)
  best_model=None
  for i in C:
    
    param = svm_parameter('-t 1  -c '+str(i)+' -d '+str(d))
    m = svm_train(prob, param)
    p_label_train,p_acc_train,p_val_train=svm_predict(train_y,train_x_p,m)
    ACC_train,s,sl=evaluations(train_y,p_label_train)
    p_label, p_acc, p_val = svm_predict(test_y, test_x_p, m)
    ACC, MSE, SCC = evaluations(test_y, p_label)
    print(ACC,ACC_train)
    if(ACC>=temp):
      temp=ACC
      maxi=i
      best_model=m
  best_models.append((best_model,maxi))



In [ ]:
h=0.1
i=2
m=best_models[i][0]
train_x_l=np.array(train_x_l)
train_y_l=np.array(train_y_l)
x_min, x_max = train_x_p[:, 0].min() - 1, train_x_p[:, 0].max() + 1
y_min, y_max = train_x_p[:, 1].min() - 1, train_x_p[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

Z,x,x=svm_predict(np.ones(xx.ravel().shape[0]),np.c_[xx.ravel(),yy.ravel()],m)
Z=np.array(Z)
Z=Z.reshape(xx.shape)
plt.figure(figsize=(7,5))
plt.contourf(xx,yy,Z,cmap=plt.cm.coolwarm,alpha=0.8)
plt.scatter(positive[:, 0], positive[:, 1], cmap=plt.cm.coolwarm,color='yellow',marker='x',s=20)
plt.scatter(negative[:, 0], negative[:, 1], cmap=plt.cm.coolwarm,color='blue',marker='o',s=20)


plt.scatter(negative_sv[:,0],negative_sv[:,1],s=20,marker="o",color="pink",cmap=plt.cm.coolwarm)
plt.scatter(positive_sv[:,0],positive_sv[:,1],s=20,marker="x",color="green",cmap=plt.cm.coolwarm)

In [ ]:
#RBF SVM with grid search on C and gamma-2D data
C=[0.001,0.03,0.6,0.1,1,30,600,1000]
gamma=[0.001,0.05,0.5,1,50,500,1000]
best_models=[]
h=dict()

prob  = svm_problem(train_y_l, train_x_l,isKernel=False)
for g in gamma:
  temp=-1
  maxi=-1
  best_model=None
  h[g]=[]
  for i in C:
    param = svm_parameter('-t 2  -c '+str(i)+' -g '+str(g))
    m = svm_train(prob, param)
    p_label_train,p_acc_train,p_val_train=svm_predict(train_y_l,train_x_l,m)
    ACC_train,s,sl=evaluations(train_y_l,p_label_train)
    p_label, p_acc, p_val = svm_predict(test_y, test_x_p, m)
    ACC, MSE, SCC = evaluations(test_y, p_label)
    h[g].append((i,ACC_train,ACC))
    print(ACC,ACC_train)
    if(ACC>=temp):
      temp=ACC
      maxi=i
      best_model=m
  best_models.append(best_model)

In [ ]:
#finds negative and postive samples, Negative and positive support vectors. Changes the value of m to run it for different models.
positive=[]
negative=[]
positive_sv=[]
negative_sv=[]
C=maxi
m=best_models[0]
best_model=m
svs=best_model.get_SV()
coefs=best_model.get_sv_coef()
train_y_l=np.array(train_y_l)
train_x_l=np.array(train_x_l)
for i in range(train_y_l.shape[0]):
  if train_y_l[i]>0:
    positive.append(train_x_l[i,:])
  else:
    negative.append(train_x_l[i,:])


print(len(coefs))
for i in range(len(coefs)):
  if coefs[i][0]>0:
    positive_sv.append(list(svs[i].values())[:-1])
  else:
    negative_sv.append(list(svs[i].values())[:-1])
positive=np.array(positive)
negative=np.array(negative)
positive_sv=np.array(positive_sv)
negative_sv=np.array(negative_sv)
print(maxi)

In [ ]:
h=0.1
i=1
m=best_models[0]
x_min, x_max = train_x_p[:, 0].min() - 1, train_x_p[:, 0].max() + 1
y_min, y_max = train_x_p[:, 1].min() - 1, train_x_p[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
# plt.subplot(2, 2, i + 1)
# plt.subplots_adjust(wspace=0.4, hspace=0.4)
Z,x,x=svm_predict(np.ones(xx.ravel().shape[0]),np.c_[xx.ravel(),yy.ravel()],m)
Z=np.array(Z)

plt.figure(figsize=(7,5))
Z=Z.reshape(xx.shape)
plt.contourf(xx,yy,Z,cmap=plt.cm.coolwarm,alpha=0.8)
plt.scatter(positive[:, 0], positive[:, 1], cmap=plt.cm.coolwarm,color='yellow',marker='x',s=20)
plt.scatter(negative[:, 0], negative[:, 1], cmap=plt.cm.coolwarm,color='blue',marker='o',s=20)

plt.scatter(negative_sv[:,0],negative_sv[:,1],s=20,marker="o",color="pink",cmap=plt.cm.coolwarm)
plt.scatter(positive_sv[:,0],positive_sv[:,1],s=20,marker="x",color="green",cmap=plt.cm.coolwarm)